In [20]:
# Installs geemap package
import subprocess
import os
import pandas as pd
import geopandas as gpd
import json #do I need this if I'm using geopandas?
import rasterio as rst

import geemap
import ee

In [21]:
import ee
import geemap

In [22]:
print(os.getcwd())
#print(os.listdir('.'))

/home/jennaguffogg/workspace/sensand/repos/jenna-test


In [23]:
#ee.Authenticate()
ee.Initialize()

geom_gpd = gpd.read_file('marysville-fire-forest-loss-test-area.geojson')
geom = geemap.geopandas_to_ee(geom_gpd)


In [131]:
#get dynamic world data
START = '2021-01-01'
END = '2022-01-01'

col_filter = ee.Filter.And(
    ee.Filter.bounds(geom),
    ee.Filter.date(START, END),
)

dw_col = ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1').filter(col_filter)
s2_col = ee.ImageCollection('COPERNICUS/S2').filter(col_filter)

# Join corresponding DW and S2 images (by system:index).
dw_s2_col = ee.Join.saveFirst('s2_img').apply(
    dw_col,
    s2_col,
    ee.Filter.equals(leftField='system:index', rightField='system:index'),
)

#dw_s2_trees_col = ee.Join.saveFirst('s2_img').apply(
#    dw_trees_col,
#    s2_col,
#    ee.Filter.equals(leftField='system:index', rightField='system:index'),
#)

# Extract an example DW image and its source S2 image.
dw_image = ee.Image(dw_s2_col.first())
s2_image = ee.Image(dw_image.get('s2_img'))

#trying a different way of getting only the trees:
# use label band because its the index of the band with the highest estimated probability
dw_trees_col_2018 = ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1').filterBounds(geom).filterDate('2018-01-01','2019-01-01').select('label').median() #can't join as this is median reduced
dw_trees_col_2022 = ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1').filterBounds(geom).filterDate('2022-01-01','2023-01-01').select('label').median()
dw_trees_image_2018 = dw_trees_col_2018.lt(2).And(dw_trees_col_2018.gt(0)).clip(geom) #the code for 'trees' is 1, so we isolate pixels with 1
dw_trees_image_2022 = dw_trees_col_2022.lt(2).And(dw_trees_col_2022.gt(0)).clip(geom)

#combine them to find spots that were trees in 2018 and aren't now
dw_trees_image_2018_2022_diff = dw_trees_col_2018.lt(2).And(dw_trees_col_2022.gt(2)).clip(geom)




In [132]:
# Create a Sentinel-2 image composite
clip_mask = geom
image_2016 = geemap.dynamic_world_s2(geom, START, END, clip=True)

# Create Dynamic World land cover composite
landcover_2018 = geemap.dynamic_world(geom, start_date='2018-01-01', end_date='2019-01-01', clip=True, return_type='visualize', reducer=None)
landcover_2022 = geemap.dynamic_world(geom, start_date='2022-01-01', end_date='2023-01-01', clip=True, return_type='visualize', reducer=None)



In [133]:
#initialise map and add layers
vis_params = {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3000}
change_vis_params = {'min':0, 'max':1, 'palette':['white','red']}
# Add legend to the map
m2 = geemap.Map()
m2.add_basemap('HYBRID')


m2.centerObject(geom)

#m2.addLayer(image, vis_params, 'Sentinel-2 image')
#m2.addLayer(landcover_2018, {}, 'Land Cover 2018')
#m2.addLayer(landcover_2022, {}, 'Land Cover 2020')

m2.addLayer(dw_trees_image_2018.updateMask(dw_trees_image_2018), change_vis_params)
m2.addLayer(dw_trees_image_2022.updateMask(dw_trees_image_2022), change_vis_params)
m2.addLayer(dw_trees_image_2018_2022_diff.updateMask(dw_trees_image_2018_2022_diff), change_vis_params)

#m2.add_legend(title="Dynamic World Land Cover", builtin_legend='Dynamic_World')
#m2.addLayer(geom, {})
m2

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [134]:
# Save Dynamic World class data in GeoTIFF format
#output_path = 'landcover.tif'
#landcover = geemap.dynamic_world(region, start_date, end_date, return_type='class')
#geemap.ee_export_image(landcover, filename=output_path, scale=10, region=region, file_per_band=False)